# Settings

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ROOT = "/content/drive/25-2-Machine-Learning-Onions/"
import sys
sys.path.append(ROOT)

In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter

from src.train_tuning import Model
from src.evaluation_multi import show_confusion_matrix, per_class_accuracy, overall_metrics, top_k_accuracy
from src.evaluation_final import OpenWorldPipeline

model_wrapper = Model(False)

# Load Data & Model

In [ ]:
with open(f"./data/preprocessed/open_train_33.pkl", "rb") as f:
    scaler_open = pickle.load(f)["scaler"]
with open(f"./data/preprocessed/close_train_33.pkl", "rb") as f:
    scaler_closed = pickle.load(f)['scaler']

with open(f"./data/preprocessed/open_test_33.pkl", "rb") as f:
    open_data = pickle.load(f)
    X_open_scaled = open_data["X"]
    y_test = open_data["y_true"]
    feat_names = open_data["feature_names"]

X_raw = scaler_open.inverse_transform(X_open_scaled)
X_multi_scaled = scaler_closed.transform(X_raw)

X_test_binary = pd.DataFrame(X_open_scaled, columns=feat_names)
X_test_multi = pd.DataFrame(X_multi_scaled, columns=feat_names)

negative_indices = np.where(y_test == -1)[0]
positive_indices = np.where(y_test != -1)[0]
sampled_neg_indices = np.random.choice(negative_indices, 30, replace=False)
balanced_indices = np.concatenate([positive_indices, sampled_neg_indices])

X_test_binary = X_test_binary.iloc[balanced_indices]
X_test_multi = X_test_multi.iloc[balanced_indices]
y_test = y_test[balanced_indices]

print("Loading complete.")
print("Test data size(binary): ", len(X_test_binary))
print("Test data size(multiclass): ", len(X_test_binary))
print("Class distribution:\n", Counter(y_test))


In [ ]:
binary_classifier = model_wrapper.load_model("./models/binary/stacked")
multi_classifier = model_wrapper.load_model("./models/multi/stacked")

In [ ]:
pipeline = OpenWorldPipeline(binary_classifier=binary_classifier, multi_classifier=multi_classifier)

# Evaluation

In [ ]:
y = y_test
y_pred = pipeline.predict(X_test_binary, X_test_multi)

n_classes = len(np.unique(y))
class_names = np.unique(y)

In [ ]:
cm = show_confusion_matrix(y, y_pred, class_names)

In [ ]:
per_class_accuracy(cm, class_names)

In [ ]:
overall_metrics(y, y_pred)

In [ ]:
y_binary = (y == -1).astype(int)
y_pred_binary = (y_pred == -1).astype(int)
overall_metrics(y_binary, y_pred_binary)

In [ ]:
mask = (y != -1) & (y_pred != -1)
y_filtered = y[mask]
y_pred_filtered = y_pred[mask]
overall_metrics(y_filtered, y_pred_filtered)